In [1]:
import os
from pyspark.sql import SparkSession
import nbimporter
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, TimestampType
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import lag 
import datetime
import time
import requests


spark = SparkSession\
        .builder \
        .appName("pyspark-notebook") \
        .master("spark://spark-master:7077") \
        .config("spark.executor.memory", "1g") \
        .config("hive.metastore.uris", "thrift://hive:9083") \
        .config("spark.sql.warehouse.dir","/opt/hive/data/warehouse") \
        .enableHiveSupport() \
        .getOrCreate()



    

24/01/11 14:42:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
from Functions.functions import read_table, create_hive_external_table

def calculating_volatility_market_cap_daily(spark):
    
    df = read_table(spark, "silver", "crypto_data")
    
   
    df = df.groupBy(F.date_format(F.col("date"), "yyyy-MM-dd").alias("date_daily"), "currency").agg(
         F.avg("priceusd").alias("priceusd_daily"),
         F.avg("priceusd_lagged").alias("priceusd_lagged_daily"),
         F.avg("circulatingsupply").alias("circulatingsupply_daily"),
         F.avg("volatility").alias("volatility_daily"),
         F.avg("volatility_percentage").alias("volatility_percentage_daily"),
         F.avg("market_cap").alias("market_cap_daily"))
    
    create_hive_external_table(spark , df, "gold", "crypto_data", "currency")




In [3]:
calculating_volatility_market_cap_daily(spark)

24/01/11 14:43:06 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [4]:
spark.stop()